### Importação de bibliotecas

In [303]:
import pandas as pd
import joblib

#### 1. Preparação do notebook

Faça a leitura do dataframe desejado, ex: month_2

In [304]:
df = pd.read_csv('../data/month_2.csv') # modifique de acordo com o desejado

Retirando as colunas indesejadas

In [305]:
df = df.drop(columns=['initialIndex', 'meterSN', 'meterSN','inputType', 'model', 'rssi', 'gatewayGeoLocation.alt', 'gatewayGeoLocation.lat', 'gatewayGeoLocation.long' ])

Adicionando a coluna de gasto

In [ ]:
df['gain'] = df['gain'].fillna(1)
df['medidor'] = df['pulseCount'] * df['gain']

df['gasto'] = 0.0

df = df.sort_values(by=['datetime', 'clientCode' ])

df['gasto'] = df.groupby(['clientCode', 'clientIndex'])['medidor'].diff()

df['gasto'] = df['gasto'].fillna(0)

df = df.sort_values(by=[ 'clientCode', 'clientIndex','datetime'])

df.head(10)

Adicionando a coluna de gasto por hora

In [ ]:

df['datetime'] = pd.to_datetime(df['datetime'])

df = df[df['gasto'] != 0]

df = df.sort_values(by=['clientCode', 'clientIndex', 'datetime'])

df['diferenca_tempo'] = df.groupby(['clientCode', 'clientIndex'])['datetime'].diff().dt.total_seconds().fillna(3600) / 3600

df['gasto_por_hora'] = round(df['gasto'] / df['diferenca_tempo'], 10)

df = df.drop(columns=['diferenca_tempo'])

df

Retirando as colunas que não são masi necessárias

In [308]:
df = df.drop(columns=['medidor', 'gain', 'meterIndex', 'pulseCount', 'datetime', 'gasto'])

In [ ]:
df.head(10)

#### 2. Dataframe de anomalias feitos por analise básica do dataframe

In [ ]:
negative_anomalies = df[df['gasto_por_hora'] < 0].copy()
negative_anomalies['tipo_anomalia'] = 1

positive_anomalies_list = []

for (clientCode, clientIndex), group_df in df.groupby(['clientCode', 'clientIndex']):
    mean = group_df['gasto_por_hora'].mean()
    std_dev = group_df['gasto_por_hora'].std()

    lower_bound = mean - 2 * std_dev
    upper_bound = mean + 2 * std_dev

    is_gasto_anomaly = (group_df['gasto_por_hora'] < lower_bound) | (group_df['gasto_por_hora'] > upper_bound)

    group_gasto_anomalies = group_df[is_gasto_anomaly].copy()
    group_gasto_anomalies['tipo_anomalia'] = 2

    positive_anomalies_list.append(group_gasto_anomalies)

positive_anomalies = pd.concat(positive_anomalies_list, ignore_index=True)

anomalies_df = pd.concat([negative_anomalies, positive_anomalies], ignore_index=True)

anomalies_df.to_csv('../data/anomalias.csv', index=False)

anomalies_df

#### 3. Dataframe de anomalias por cliente feito por analise básica

In [ ]:
df_cliente_anomalies = anomalies_df.groupby(['clientCode', 'clientIndex']).size().reset_index(name='total_de_anomalias')

for tipo in range(1, 3):
    anomalias_tipo = anomalies_df[anomalies_df['tipo_anomalia'] == tipo].groupby(['clientCode', 'clientIndex']).size().reset_index(name=f'anomalia_tipo_{tipo}')
    
    df_cliente_anomalies = df_cliente_anomalies.merge(anomalias_tipo, on=['clientCode', 'clientIndex'], how='left')

df_cliente_anomalies.fillna(0, inplace=True)

df_cliente_anomalies = df_cliente_anomalies.sort_values(by=['anomalia_tipo_1', 'anomalia_tipo_2'], ascending=[False, False])

gasto_count = df.groupby(['clientCode', 'clientIndex']).size().reset_index(name='quantidade_dados')

df_cliente_anomalies = df_cliente_anomalies.merge(gasto_count, on=['clientCode', 'clientIndex'], how='left')

df_cliente_anomalies['porcentagem_anomalia'] = df_cliente_anomalies['total_de_anomalias'] / df_cliente_anomalies['quantidade_dados']

df_cliente_anomalies.to_csv('../data/anomalias_por_cliente.csv', index=False)

df_cliente_anomalies

#### Dataframe de clientes anômalos

Pegando somente os pontos com gastos positivos

In [313]:
df = df[df['gasto_por_hora'] >= 0]

Agrupando por clientCode e clientIndex.

In [314]:
df = df.groupby(['clientCode', 'clientIndex']).agg(
    media_gasto=('gasto_por_hora', 'mean'),
    max_gasto=('gasto_por_hora', 'max'),
    min_gasto=('gasto_por_hora', 'min'),
    desvio_padrao=('gasto_por_hora', 'std')
).reset_index()

Retirando os valores NA

In [315]:
df = df.dropna()

Separando o dataframe para rodar no modelo

In [316]:
df_sem_id = df.copy()
df_sem_id = df.drop(columns=['clientCode', 'clientIndex'])

Carregando o modelo

In [317]:
svc_model_loaded = joblib.load('../data/svc_model.pkl')

Adicionando a coluna de cluster de acordo com o que foi previsto

In [318]:
cluster_pred = svc_model_loaded.predict(df_sem_id)

df['cluster'] = cluster_pred

Adicionando a coluna de anomalia de acordo com os clusters

In [ ]:
df['anomaly'] = ((df['cluster'] == 5) | (df['cluster'] == 6)).astype(int)

df

Salva o dataframe

In [320]:
df.to_csv('../data/anomaly_cliente.csv') #modifique o nome de acordo com o desejado